In [26]:
import os
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np
from pprint import pprint as pp
import warnings
warnings.filterwarnings('ignore')

## Human in the Loop
In the previous chapter, you perfected your knowledge of the standard supervised learning workflows. In this chapter, you will critically examine the ways in which expert knowledge is incorporated in supervised learning. This is done through the identification of the appropriate unit of analysis which might require feature engineering across multiple data sources, through the sometimes imperfect process of labeling examples, and through the specification of a loss function that captures the true business value of errors made by your machine learning model.

In [4]:
class PDF(object):
    def __init__(self, pdf, size=(1080,720)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return f'<iframe src={self.pdf} width={self.size[0]} height={self.size[1]}></iframe>'

    def _repr_latex_(self):
        return fr'\includegraphics[width=1.0\textwidth]{{{self.pdf}}}'

In [3]:
PDF('pdf/chapter2.pdf',size=(1080, 720))

# "Expert Knowledge"

In [14]:
flows = pd.read_csv('data/lanl_flows.csv')

<h1 class="exercise--title">Is the source or the destination bad?</h1><div class=""><p>In the previous lesson, you used the <em>destination</em> computer as your entity of interest. However, your cybersecurity analyst just told you that it is the infected machines that generate the bad traffic, and will therefore appear as a <em>source</em>, not a destination, in the <code>flows</code> dataset. </p>
<p>The data <code>flows</code> has been preloaded, as well as the list <code>bad</code> of infected IDs and the feature extractor <code>featurizer()</code> from the previous lesson. You also have <code>numpy</code> available as <code>np</code>, <code>AdaBoostClassifier()</code>, and <code>cross_val_score()</code>.</p></div>

In [15]:
def featurize(df):
    return {
        'unique_ports': len(set(df['destination_port'])),
        'average_packet': np.mean(df['packet_count']),
        'average_duration': np.mean(df['duration'])
    } 

In [16]:
bads = {'C1', 'C10', 'C10005', 'C1003', 'C1006', 'C1014', 'C1015', 'C102', 'C1022', 'C1028', 'C10405', 'C1042', 'C1046', 'C10577', 'C1065', 'C108', 'C10817', 'C1085', 'C1089', 'C1096', 'C11039', 'C11178', 'C1119', 'C11194', 'C1124', 'C1125', 'C113', 'C115',  'C11727', 'C1173', 'C1183', 'C1191', 'C12116', 'C1215', 'C1222', 'C1224', 'C12320', 'C12448', 'C12512', 'C126', 'C1268', 'C12682', 'C1269', 'C1275', 'C1302', 'C1319', 'C13713', 'C1382', 'C1415', 'C143', 'C1432', 'C1438', 'C1448', 'C1461', 'C1477', 'C1479', 'C148', 'C1482', 'C1484', 'C1493', 'C15', 'C1500', 'C1503', 'C1506', 'C1509', 'C15197', 'C152', 'C15232', 'C1549', 'C155', 'C1555', 'C1567', 'C1570', 'C1581', 'C16088', 'C1610', 'C1611', 'C1616', 'C1626', 'C1632', 'C16401', 'C16467', 'C16563', 'C1710', 'C1732', 'C1737', 'C17425', 'C17600', 'C17636', 'C17640', 'C17693', 'C177', 'C1776', 'C17776', 'C17806', 'C1784', 'C17860', 'C1797', 'C18025', 'C1810', 'C18113', 'C18190', 'C1823', 'C18464', 'C18626', 'C1887', 'C18872', 'C19038', 'C1906', 'C19156', 'C19356', 'C1936', 'C1944', 'C19444', 'C1952', 'C1961', 'C1964', 'C1966', 'C1980', 'C19803', 'C19932', 'C2012', 'C2013', 'C20203', 'C20455', 'C2057', 'C2058', 'C20677', 'C2079', 'C20819', 'C2085', 'C2091', 'C20966', 'C21349', 'C21664', 'C21814', 'C21919', 'C21946', 'C2196', 'C21963', 'C22174', 'C22176', 'C22275', 'C22409', 'C2254', 'C22766', 'C231', 'C2341', 'C2378', 'C2388', 'C243', 'C246', 'C2519', 'C2578', 'C2597', 'C2604', 'C2609', 'C2648', 'C2669', 'C2725', 'C2816', 'C2844', 'C2846', 'C2849', 'C2877', 'C2914', 'C294', 'C2944', 'C3019', 'C302', 'C3037', 'C305', 'C306', 'C307', 'C313', 'C3153', 'C3170', 'C3173', 'C3199', 'C3249', 'C3288', 'C3292', 'C3303', 'C3305', 'C332', 'C338', 'C3380', 'C3388', 'C3422', 'C3435', 'C3437', 'C3455', 'C346', 'C3491', 'C3521', 'C353', 'C3586', 'C359', 'C3597', 'C3601', 'C3610', 'C3629', 'C3635', 'C366', 'C368', 'C3699', 'C370', 'C3755', 'C3758', 'C3813', 'C385', 'C3888', 'C395', 'C398', 'C400', 'C4106', 'C4159', 'C4161', 'C42', 'C423', 'C4280', 'C429', 'C430', 'C4403', 'C452', 'C4554', 'C457', 'C458', 'C46', 'C4610', 'C464', 'C467', 'C477', 'C4773', 'C4845', 'C486', 'C492', 'C4934', 'C5030', 'C504', 'C506', 'C5111', 'C513', 'C52', 'C528', 'C529', 'C5343', 'C5439', 'C5453', 'C553', 'C5618', 'C5653', 'C5693', 'C583', 'C586', 'C61', 'C612', 'C625', 'C626', 'C633', 'C636', 'C6487', 'C6513', 'C685', 'C687', 'C706', 'C7131', 'C721', 'C728', 'C742', 'C7464', 'C7503', 'C754', 'C7597', 'C765', 'C7782', 'C779', 'C78', 'C791', 'C798', 'C801', 'C8172', 'C8209', 'C828', 'C849', 'C8490', 'C853', 'C8585', 'C8751', 'C881', 'C882', 'C883', 'C886', 'C89', 'C90', 'C9006', 'C917', 'C92', 'C923', 'C96', 'C965', 'C9692', 'C9723', 'C977', 'C9945'}


In [17]:
# Group by source computer, and apply the feature extractor
out = flows.groupby('source_computer').apply(featurize)

# Convert the iterator to a dataframe by calling list on it
X = pd.DataFrame(list(out), index=out.index)

# Check which sources in X.index are bad to create labels
y = [x in bads for x in X.index]

In [18]:
X.head()

,unique_ports,average_packet,average_duration
source_computer,,,
C10,4,222.000000,5.000000
C10026,2,21.000000,39.000000
C10047,5,21.076923,7.538462
C1015,35,5.371429,27.571429
C10235,1,11.000000,0.000000


In [19]:
# Report the average accuracy of Adaboost over 3-fold CV

print(np.mean(cross_val_score(AdaBoostClassifier(), X, y)))

0.9428571428571428


<h1 class="exercise--title">Feature engineering on grouped data</h1><div class=""><p>You will now build on the previous exercise, by considering one additional feature: the number of unique protocols used by each source computer. Note that with grouped data, it is always possible to construct features in this manner: you can take the number of unique elements of all categorical columns, and the mean of all numeric columns as your starting point. As before, you have <code>flows</code> preloaded, <code>cross_val_score()</code> for measuring accuracy, <code>AdaBoostClassifier()</code>, <code>pandas</code> as <code>pd</code> and <code>numpy</code> as <code>np</code>.</p></div>

In [20]:
# Create a feature counting unique protocols per source
protocols = flows.groupby('source_computer').apply(lambda df: len(set(df.protocol)))

# Convert this feature into a dataframe, naming the column
protocols_DF = pd.DataFrame(protocols, index=protocols.index, columns=['protocol'])

# Now concatenate this feature with the previous dataset, X
X_more = pd.concat([X, protocols_DF], axis=1)

# Refit the classifier and report its accuracy
print(np.mean(cross_val_score(AdaBoostClassifier(), X_more, y)))

0.9428571428571428


<h1 class="exercise--title">Turning a heuristic into a classifier</h1><div class=""><p>You are surprised by the fact that heuristics can be so helpful. So you decide to treat the heuristic that "too many unique ports is suspicious" as a classifier in its own right. You achieve that by thresholding the number of unique ports per source by the average number used in bad source computers -- these are computers for which the label is <code>True</code>. The dataset is preloaded and split into training and test, so you have objects <code>X_train</code>, <code>X_test</code>, <code>y_train</code> and <code>y_test</code> in memory. Your imports include <code>accuracy_score()</code>, and <code>numpy</code> as <code>np</code>. To clarify: you won't be fitting a classifier from scikit-learn in this exercise, but instead you will define your own classification rule explicitly!</p></div>

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [22]:
#Create a new dataset `X_train_bad` by subselecting bad hosts
X_train_bad = X_train[y_train]

#Calculate the average of `unique_ports` in bad examples

avg_bad_ports = np.mean(X_train_bad.unique_ports)

#Label as positive sources that use more ports than that

pred_port = X_test['unique_ports'] > avg_bad_ports

#Print the `accuracy_score` of the heuristic

print(accuracy_score(y_test, pred_port))

0.9261744966442953


<h1 class="exercise--title">Combining heuristics</h1><div class=""><p>A different cyber analyst tells you that during certain types of attack, the infected source computer sends small bits of traffic, to avoid detection. This makes you wonder whether it would be better to create a combined heuristic that simultaneously looks for large numbers of ports and small packet sizes. Does this improve performance over the simple port heuristic? As with the last exercise, you have <code>X_train</code>, <code>X_test</code>, <code>y_train</code> and <code>y_test</code> in memory. The sample code also helps you reproduce the outcome of the port heuristic, <code>pred_port</code>. You also have <code>numpy</code> as <code>np</code> and <code>accuracy_score()</code> preloaded.</p></div>

In [24]:
# Compute the mean of average_packet for bad sources
avg_bad_packet = np.mean(X_train[y_train]['average_packet'])

# Label as positive if average_packet is lower than that
pred_packet = X_test['average_packet'] < avg_bad_packet

# Find indices where pred_port and pred_packet both True
pred_port = X_test['unique_ports'] > avg_bad_ports
pred_both = pred_packet & pred_port

# Ports only produced an accuracy of 0.919. Is this better?
print(accuracy_score(y_test, pred_both))

0.9328859060402684


<h1 class="exercise--title">Dealing with label noise</h1><div class=""><p>One of your cyber analysts informs you that many of the labels for the first 100 source computers in your training data might be wrong because of a database error. She hopes you can still use the data because most of the labels are still correct, but asks you to treat these 100 labels as "noisy". Thankfully you know how to do that, using weighted learning. The contaminated data is available in your workspace as <code>X_train</code>, <code>X_test</code>, <code>y_train_noisy</code>, <code>y_test</code>. You want to see if you can improve the performance of a <code>GaussianNB()</code> classifier using weighted learning. You can use the optional parameter <code>sample_weight</code>, which is supported by the <code>.fit()</code> methods of most popular classifiers. The function <code>accuracy_score()</code> is preloaded. You can consult the image below for guidance. </p>
<p><img src="https://assets.datacamp.com/production/repositories/3554/datasets/ea99ce2b5baa3cb9f3d9085b7387f2ea7d3bdfc8/wsl_noisy_labels.png" alt=""></p></div>

In [38]:
y_train_noisy = y_train.copy()

In [40]:
for i in range(100):
    y_train_noisy[i] = True

In [45]:
from sklearn.naive_bayes import GaussianNB

# Fit a Gaussian Naive Bayes classifier to the training data
clf = GaussianNB().fit(X_train, y_train_noisy)

# Report its accuracy on the test data
print(accuracy_score(y_test, clf.predict(X_test)))

# Assign half the weight to the first 100 noisy examples
weights = [0.5]*100 + [1.0]*(len(X_train)-100)

# Refit using weights and report accuracy. Has it improved?
clf_weights = GaussianNB().fit(X_train, y_train_noisy, sample_weight=weights)
print(accuracy_score(y_test, clf_weights.predict(X_test)))

0.06711409395973154
0.06711409395973154


# 3. Model Lifecycle Management
In the previous chapter, you employed different ways of incorporating feedback from experts in your workflow, and evaluating it in ways that are aligned with business value. Now it is time for you to practice the skills needed to productize your model and ensure it continues to perform well thereafter by iteratively improving it. You will also learn to diagnose dataset shift and mitigate the effect that a changing environment can have on your model's accuracy. 

In [6]:
PDF('pdf/chapter3.pdf')

In [7]:
df = pd.read_csv('data/arrh.csv')

In [12]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

<h1 class="exercise--title">Your first pipeline - again!</h1><div class=""><p>Back in the arrhythmia startup, your monthly review is coming up, and as part of that an expert Python programmer will be reviewing your code. You decide to tidy up by following best practices and replace your script for feature selection and random forest classification, with a pipeline. You are using a training dataset available as <code>X_train</code> and <code>y_train</code>, and a number of modules: <code>RandomForestClassifier</code>, <code>SelectKBest()</code> and <code>f_classif()</code> for feature selection, as well as <code>GridSearchCV</code> and <code>Pipeline</code>.</p></div>

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
# Create pipeline with feature selector and classifier
pipe = Pipeline([
    ('feature_selection', SelectKBest(f_classif)),
    ('clf', rf(random_state=2))])

# Create a parameter grid
params = {
   'feature_selection__k':[10,20],
    'clf__n_estimators':[2, 5]}

# Initialize the grid search object
grid_search = GridSearchCV(pipe, param_grid=params)

# Fit it to the data and print the best value combination
print(grid_search.fit(X_train, y_train).best_params_)

{'clf__n_estimators': 5, 'feature_selection__k': 10}


<h1 class="exercise--title">Custom scorers in pipelines</h1><div class=""><p>You are proud of the improvement in your code quality, but just remembered that previously you had to use a custom scoring metric in order to account for the fact that false positives are costlier to your startup than false negatives. You hence want to equip your pipeline with scorers other than accuracy, including <code>roc_auc_score()</code>, <code>f1_score()</code>, and you own custom scoring function. The pipeline from the previous lesson is available as <code>pipe</code>, as is the parameter grid as <code>params</code> and the training data as <code>X_train</code>, <code>y_train</code>. You also have <code>confusion_matrix()</code> for the purpose of writing your own metric.</p></div>

In [27]:
from sklearn.metrics import make_scorer, roc_auc_score
# Create a custom scorer
scorer = make_scorer(roc_auc_score)

# Initialize the CV object
gs = GridSearchCV(pipe, param_grid=params, scoring=scorer)

# Fit it to the data and print the winning combination
print(gs.fit(X_train, y_train).best_params_)

{'clf__n_estimators': 5, 'feature_selection__k': 10}


In [29]:
from sklearn.metrics import f1_score
# Create a custom scorer
scorer = make_scorer(f1_score)

# Initialize the CV object
gs = GridSearchCV(pipe, param_grid=params, scoring=scorer)

# Fit it to the data and print the winning combination
print(gs.fit(X_train, y_train).best_params_)

{'clf__n_estimators': 5, 'feature_selection__k': 10}


In [32]:
from sklearn.metrics import confusion_matrix
def my_metric(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return cost_fp * fp + cost_fn * fn

In [34]:
from sklearn.metrics import f1_score
# Create a custom scorer
scorer = make_scorer(my_metric)

# Initialize the CV object
gs = GridSearchCV(pipe, param_grid=params, scoring=scorer)

# Fit it to the data and print the winning combination
print(gs.fit(X_train, y_train).best_params_)

{'clf__n_estimators': 5, 'feature_selection__k': 20}


<h1 class="exercise--title">Pickles</h1><div class=""><p>Finally, it is time for you to push your first model to production. It is a random forest classifier which you will use as a baseline, while you are still working to develop a better alternative. You have access to the data split in training test with their usual names, <code>X_train</code>, <code>X_test</code>, <code>y_train</code> and <code>y_test</code>, as well as to the modules <code>RandomForestClassifier()</code> and <code>pickle</code>, whose methods <code>.load()</code> and <code>.dump()</code> you will need for this exercise.</p></div>

In [38]:
import pickle
# Fit a random forest to the training set
clf = rf(random_state=42).fit(
  X_train, y_train)

# Save it to a file, to be pushed to production
with open('model.pkl', 'wb') as file:
    pickle.dump(clf, file=file)

# Now load the model from file in the production environment
with open('model.pkl', 'rb') as file:
    clf_from_file = pickle.load(file)

# Predict the labels of the test dataset
preds = clf_from_file.predict(X_test)

<h1 class="exercise--title">Custom function transformers in pipelines</h1><div class=""><p>At some point, you were told that the sensors might be performing poorly for obese individuals. Previously you had dealt with that using weights, but now you are thinking that this information might also be useful for feature engineering, so you decide to replace the recorded weight of an individual with an indicator of whether they are obese. You want to do this using pipelines. You have <code>numpy</code> available as <code>np</code>, <code>RandomForestClassifier()</code>, <code>FunctionTransformer()</code>, and <code>GridSearchCV()</code>.</p></div>

In [43]:
from sklearn.preprocessing import FunctionTransformer
# Define a feature extractor to flag very large values
def more_than_average(X, multiplier=1.0):
    Z = X.copy()
    Z[:,1] = Z[:,1] > multiplier*np.mean(Z[:,1])
    return Z

# Convert your function so that it can be used in a pipeline
pipe = Pipeline([
  ('ft', FunctionTransformer(more_than_average)),
  ('clf', RandomForestClassifier(random_state=2))])

# Optimize the parameter multiplier using GridSearchCV
params = {'ft__multiplier': [1,2,3]}
gs = GridSearchCV(pipe, param_grid=params)
print(gs.fit(X_train, y_train).best_params_)

ValueError: Invalid parameter multiplier for estimator FunctionTransformer(accept_sparse=False, check_inverse=True,
                    func=<function more_than_average at 0x7fef04756c10>,
                    inv_kw_args=None, inverse_func=None, kw_args=None,
                    validate=False). Check the list of available parameters with `estimator.get_params().keys()`.